# 目标检测 Mask R-CNN

本文主要参考了pytorch官方文档：[TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)，了解目标检测的实践，后面会有能用到本专业内容上的。关于算法方面的基础只是可以参考elks项目中的记录。

在Penn-Fudan数据库中对行人检测和分割的预训练Mask R-CNN模型进行微调。它包含170张图像，其中包含345个行人实例，我们将用它来说明如何在torchvision中使用新功能，以便在自定义数据集上训练实例细分模型。